In [1]:
import codecs
import re
import pandas as pd
import numpy as np
from numpy import log,min

In [2]:
data_path = r'E:\data\NLP\book\tianlongbabu.txt'
f = codecs.open(data_path,'r','utf-8')
s = f.read()
s = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）()“”：‘’；:]+", "",s)
s = re.sub("[a-zA-Z0-9]","",s)

In [3]:
s = s[1:]
s[:3000]

'天龙八部一少年游青衫磊落险峰行玉璧月华明马疾香幽崖高人远微步觳纹生谁家子弟谁家院无计悔多情虎啸龙吟换巢鸾凤剑气碧烟横二苏幕遮向来痴从此醉水榭听香指点群豪戏剧饮千杯男儿事杏子林中商略平生义昔时因今日意胡汉恩仇须倾英雄泪虽万千人吾往矣悄立雁门绝壁无余子三破阵子千里茫茫若梦双眸粲粲如星塞上牛羊空许约烛畔鬓云有旧盟莽苍踏雪行赤手屠熊搏虎金戈荡寇鏖兵草木残生颅铸铁虫豸凝寒掌作冰挥洒缚豪英四洞仙歌输赢成败又争由人算且自逍遥没谁管奈天昏地暗斗转星移风骤紧缥缈峰头云乱红颜弹指老刹那芳华梦里真真语真幻同一笑到头万事俱空胡涂醉情长计短解不了名缰系嗔贪却试问几时把痴心断五水龙吟燕云十八飞骑奔腾如虎风烟举老魔小丑岂堪一击胜之不武王霸雄图血海深仇尽归尘土念枉求美眷良缘安在枯井底暗泥处酒罢问君三语为谁开茶花满路王孙落魄怎生消得杨枝玉露敝屣荣华浮云生死此身何惧教单于折剪六军辟易奋英雄怒一青衫磊落险峰行青光闪动一柄青钢剑倏地刺出指向在年汉子左肩使剑少年不等招用老腕抖剑斜剑锋已削向那汉子右颈那中年汉子剑挡格铮的一声响双剑相击嗡嗡作声震声未绝双剑剑光霍霍已拆了三招中年汉子长剑猛地击落直砍少年顶门那少年避向右侧左手剑诀一引青钢剑疾刺那汉子大腿两人剑法迅捷全力相搏练武厅东坐着二人上首是个四十左右的中年道姑铁青着脸嘴唇紧闭下首是个五十余岁的老者右手捻着长须神情甚是得意两人的座位相距一丈有余身后各站着二十余名男女弟子西边一排椅子上坐着十余位宾客东西双方的目光都集注于场中二人的角斗眼见那少年与中年汉子已拆到七十余招剑招越来越紧兀自未分胜败突然中年汉子一剑挥出用力猛了身子微微一幌似欲摔跌西边宾客中一个身穿青衫的年轻男子忍不住嗤的一声笑他随即知道失态忙伸手按住了口便在这时场中少年左手呼一掌拍出击向那汉子后心那汉子向前跨出一步避开手中长剑蓦地圈转喝一声着那少年左腿已然中剑腿下一个踉跄长剑在地下一撑站直身子待欲再斗那中年汉子已还剑入鞘笑道褚师弟承让承让伤得不厉害么那少年脸色苍白咬着嘴唇道多谢龚师兄剑下留情那长须老者满脸得色微微一笑说道东宗已胜了三阵看来这剑湖宫又要让东宗再住五年了辛师妹咱们还须比下去么坐在他上首的那中年道姑强忍怒气说道左师果然调教得好徒儿但不知左师兄对无量玉壁的钻研这五年来可已大有心得么长须老者向她瞪了一眼正色道师妹怎地忘了本派的规矩那道姑哼了一声便不再说下去了这老者姓左名叫子穆是无量剑东宗的掌门那道

In [4]:
#为了方便调用，自定义了一个正则表达式的词典
myre = {2:'(..)', 3:'(...)', 4:'(....)', 5:'(.....)', 6:'(......)', 7:'(.......)'}

min_count = 10 #录取词语最小出现次数
min_support = 30 #录取词语最低聚合度，1代表着随机组合
min_s = 3 #录取词语最低信息熵，越大说明越有可能独立成词
max_sep = 4 #候选词语的最大字数
t=[] #保存结果用。

t.append(pd.Series(list(s)).value_counts()) #逐字统计
tsum = t[0].sum() #统计总字数
rt = [] #保存结果用

for m in range(2, max_sep+1):
    print(u'正在生成%s字词...'%m)
    t.append([])
    for i in range(m): #生成所有可能的m字词
        t[m-1] = t[m-1] + re.findall(myre[m], s[i:])
    
    t[m-1] = pd.Series(t[m-1]).value_counts() #逐词统计
    t[m-1] = t[m-1][t[m-1] > min_count] #最小次数筛选
    tt = t[m-1]
    for k in range(m-1):
        qq = np.array(list(map(lambda ms: tsum*t[m-1][ms]/t[m-2-k][ms[:m-1-k]]/t[k][ms[m-1-k:]], tt.index))) > min_support #最小聚合度筛选。
        tt = tt[qq]
    rt.append(tt.index)
#     print(tt.index[:10])
def cal_S(sl): #信息熵计算函数
    return -((sl/sl.sum()).apply(log)*sl/sl.sum()).sum()

正在生成2字词...
正在生成3字词...
正在生成4字词...


In [5]:
for i in range(2, max_sep+1):
    print(u'正在进行%s字词的最大熵筛选(%s)...'%(i, len(rt[i-2])))
    pp = [] #保存所有的左右邻结果
    for j in range(i+2):
        pp = pp + re.findall('(.)%s(.)'%myre[i], s[j:])
    pp = pd.DataFrame(pp).set_index(1).sort_index() #先排序，这个很重要，可以加快检索速度
    index = np.sort(np.intersect1d(rt[i-2], pp.index)) #作交集
    #下面两句分别是左邻和右邻信息熵筛选
    index = index[np.array(list(map(lambda s: cal_S(pd.Series(pp[0][s]).value_counts()), index))) > min_s]
    rt[i-2] = index[np.array(list(map(lambda s: cal_S(pd.Series(pp[2][s]).value_counts()), index))) > min_s]

正在进行2字词的最大熵筛选(3561)...
正在进行3字词的最大熵筛选(2274)...
正在进行4字词的最大熵筛选(1069)...


In [6]:
#下面都是输出前处理
for i in range(len(rt)):
    t[i+1] = t[i+1][rt[i]]
    t[i+1] = t[i+1].sort_values(ascending = False)

In [7]:
res_path = r'E:\data\NLP\book\result.txt'
# res = pd.DataFrame({'2':t[1].index,'2_num':t[1].values,'3':t[2].index,'3_num':t[2].values,'4':t[3].index,'4_num':t[3].values})
res = pd.DataFrame()
for i in range(2,max_sep+1):
    id = str(i)
    num = id+'num'
    res[id] = t[i-1].index[:20]
    res[num] = t[i-1].values[:20]

In [8]:
res

,2,2num,3,3num,4,4num
0,段誉,3378,慕容复,922,南海鳄神,499
1,什么,2533,王语嫣,865,慕容公子,225
2,萧峰,1794,段正淳,741,耶律洪基,175
3,虚竹,1635,木婉清,739,六脉神剑,147
4,自己,1628,鸠摩智,561,站起身来,126
5,阿紫,1135,游坦之,515,啊的一声,107
6,乔峰,1118,有什么,479,这几句话,105
7,武功,1073,丁春秋,433,星宿老怪,105
8,姑娘,990,包不同,408,大吃一惊,102
9,阿朱,985,少林寺,382,吃了一惊,95


In [51]:
from tqdm import tqdm
er = []
for i in range(2):
    er = er + re.findall(myre[2],s[i:])
er = pd.Series(er).value_counts()
cut = list(s)
num_cut = 0
for i in tqdm(range(10000-1)):
    if tsum*er[s[i:i+2]]/t[0][s[i]]/t[0][s[i+1]] < 4:
        cut.insert(i+1+num_cut,' ')
        num_cut = num_cut +1
cut = "".join(cut)
cut = cut.split(' ')
cut[:100]


100%|████████████████████████████████████████████████████████████████████████████| 9999/9999 [00:04<00:00, 2231.31it/s]

['天龙八部',
 '一',
 '少年',
 '游',
 '青衫磊落险',
 '峰',
 '行',
 '玉璧月华明',
 '马疾香幽崖高',
 '人',
 '远',
 '微步觳纹生',
 '谁家',
 '子弟',
 '谁家院',
 '无计悔多',
 '情虎啸龙吟换巢鸾凤剑气',
 '碧烟横',
 '二',
 '苏幕遮向来',
 '痴从此醉水榭听香指点',
 '群豪戏剧饮千杯男儿',
 '事杏子',
 '林中商略平生',
 '义昔时',
 '因',
 '今日',
 '意',
 '胡汉恩仇须倾英雄泪虽',
 '万千',
 '人吾往矣悄立雁门绝壁',
 '无',
 '余',
 '子',
 '三',
 '破阵',
 '子',
 '千里茫茫若梦双眸粲粲如',
 '星塞上',
 '牛羊空许约烛畔鬓云',
 '有旧盟莽苍踏雪行赤手屠熊搏虎金戈荡寇鏖兵草木残生颅铸铁虫豸凝寒掌',
 '作冰挥洒缚豪英',
 '四',
 '洞仙歌输赢成败又争由',
 '人',
 '算且',
 '自逍遥没',
 '谁管奈天昏地',
 '暗斗转星移风骤紧缥缈峰',
 '头',
 '云乱',
 '红颜弹指',
 '老刹那芳华梦里',
 '真',
 '真',
 '语',
 '真幻同',
 '一笑',
 '到',
 '头',
 '万',
 '事俱空胡涂醉情长计短解',
 '不',
 '了',
 '名缰系嗔贪却试问',
 '几时',
 '把痴心',
 '断五',
 '水龙吟燕云十八飞骑奔腾如虎风烟举',
 '老魔小丑岂堪一',
 '击胜之',
 '不',
 '武',
 '王霸雄图血海深仇尽归尘土念枉求美眷良缘安',
 '在枯井底暗泥处',
 '酒罢问君三',
 '语',
 '为',
 '谁',
 '开茶花满路',
 '王孙落魄怎生消',
 '得杨枝玉露敝屣荣华浮云生死',
 '此',
 '身',
 '何惧教单于折剪六军辟易奋英雄怒',
 '一',
 '青衫磊落险',
 '峰',
 '行',
 '青光闪动',
 '一柄青钢剑倏地',
 '刺出',
 '指向',
 '在',
 '年汉子',
 '左肩使剑',
 '少年',
 '不',
 '等',
 '招']

In [52]:
cut[400:500]

['东西',
 '两宗',
 '也不通音问',
 '三十五年来',
 '东西',
 '二宗互',
 '有',
 '胜负东宗胜过',
 '四',
 '次',
 '西宗胜过',
 '两次',
 '那龚姓',
 '中年汉子',
 '与褚姓',
 '少年',
 '相斗',
 '已',
 '是',
 '本',
 '次比剑',
 '中',
 '的',
 '第四场姓龚',
 '的',
 '汉子既',
 '胜东宗',
 '四赛三',
 '胜',
 '第五场便',
 '不用',
 '比',
 '了',
 '西首锦凳上所坐',
 '的',
 '则是',
 '别派人',
 '士',
 '其中',
 '有',
 '的',
 '是',
 '东西',
 '二宗掌门人共同',
 '出',
 '面邀请',
 '的',
 '公证',
 '人',
 '其余',
 '则是',
 '前来',
 '观礼',
 '的',
 '嘉宾',
 '这些人都是',
 '云南',
 '武林中',
 '的',
 '知',
 '名',
 '之士',
 '只',
 '坐在',
 '最',
 '下首',
 '的',
 '那个青衣',
 '少年',
 '却是个',
 '无名',
 '之辈偏',
 '是',
 '他',
 '在龚姓',
 '汉子',
 '伴作',
 '失足',
 '时嗤的',
 '一声',
 '笑',
 '这',
 '少年乃随滇南普洱老',
 '武',
 '师',
 '马五德',
 '而来',
 '马五德',
 '是',
 '大',
 '茶商豪富好',
 '客颇有孟尝之',
 '风',
 '江湖上',
 '落魄的武',
 '师',
 '前去投奔',
 '他',
 '必竭诚相待因此人',
 '缘甚佳武功却是']

In [23]:
er

'天龙八部一少年游青衫磊落险峰行玉璧月华明马疾香幽崖高人远微步觳纹生谁家子弟谁家院无计悔多情虎啸龙吟换巢鸾凤剑气碧烟横二苏幕遮向来痴从此醉水榭听香指点群豪戏剧饮千杯男儿事杏子林中商略平生义昔时因今日意胡'